# Self-contained Robo-advisor

In [3]:
import pandas as pd
import plotly.graph_objects as go

from PortfolioPredictor import extract_asset_returns, chain_portfolio_performance, evaluate_portfolios_over_time
from fetchData import fetch_raw_data_yf, getSNP500, getNasdaq_comp, fetch_raw_data_yf_all
from LearningRBA import MLRBA_V2
from PortfolioFunction import get_matrices

In [4]:
date_ranges = [
    (pd.Timestamp("2021-01-01"), pd.Timestamp("2023-01-01")),
]

for idx, (start_date, end_date) in enumerate(date_ranges, start=1):
    investment_period = 730
    investment_end_date = end_date + pd.Timedelta(days=investment_period)

    assets = getSNP500()
    raw_data, _, _ = fetch_raw_data_yf(assets, start_date, investment_end_date)

    investments = evaluate_portfolios_over_time(raw_data, end_date, window_size=5, length_of_investment=None,candidates_per_divison=3, candidates_divison=4
    )

    names, annualized_returns, _, _, _, cov, correlation_matrix = get_matrices(raw_data[:end_date])
    _, initial_portfolio, _, _, _ = MLRBA_V2(names, cov, annualized_returns, correlation_matrix)

    ML_portfolio = []
    portfolio_holdings = []
    initial_value = 100

    for i in range(len(investments)):
        curr_best_portfolio = investments[i]['portfolio']
        p_start_date = investments[i]['start_date']
        p_end_date = investments[i]['end_date']

        if not curr_best_portfolio:
            previous_value = ML_portfolio[-1].iloc[-1] if i > 0 else initial_value
            window_index = raw_data.loc[p_start_date:p_end_date].index
            portfolio_daily_returns = pd.Series(previous_value, index=window_index)
            held_tickers = []
        else:
            best_curr_port_assets = curr_best_portfolio['tickers']
            best_curr_port_assets_test_data = extract_asset_returns(raw_data, best_curr_port_assets, p_start_date, p_end_date)
            curr_best_portfolio_weights = curr_best_portfolio['weights']
            weighted_returns = best_curr_port_assets_test_data.mul(curr_best_portfolio_weights, axis='columns')
            portfolio_daily_returns = weighted_returns.sum(axis=1)
            held_tickers = best_curr_port_assets

        ML_portfolio.append(portfolio_daily_returns)
        portfolio_holdings.append((p_start_date, p_end_date, held_tickers))

    ML_portfolio_streamed = chain_portfolio_performance(ML_portfolio, starting_value=initial_value)

        # Print tickers held at each point
    for period_info in portfolio_holdings:
        start, end, tickers = period_info
        print(f"Held from {start} to {end}: {tickers}")

    ML_daily_returns = ML_portfolio_streamed.pct_change()
    ML_cumulative_returns = (1 + ML_daily_returns).cumprod()
    ML_cumulative_returns.iloc[0] = 1
    ML_portfolio_normalized = (ML_cumulative_returns / ML_cumulative_returns.iloc[0]) * 100

    best_port_assets = initial_portfolio['tickers']
    best_port_assets_test_data = raw_data[end_date:].loc[:, best_port_assets]

    Nasdaq_comp = getNasdaq_comp(ML_cumulative_returns.index[0], ML_cumulative_returns.index[-1])

    best_portfolio_weights = initial_portfolio['weights']
    normalized_prices = best_port_assets_test_data.div(best_port_assets_test_data.iloc[0])
    daily_returns = normalized_prices.pct_change()
    weighted_returns = daily_returns.mul(best_portfolio_weights, axis='columns')
    portfolio_daily_returns = weighted_returns.sum(axis=1)
    portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()
    portfolio_start = portfolio_cumulative_returns.iloc[0]
    portfolio_normalized = (portfolio_cumulative_returns / portfolio_start) * 100

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=Nasdaq_comp.index,
        y=Nasdaq_comp['Normalized'],
        mode='lines',
        name='Nasdaq Composite'
    ))

    fig.add_trace(go.Scatter(
        x=ML_cumulative_returns.index,
        y=ML_portfolio_normalized,
        mode='lines',
        name='Portfolio with Adjustments'
    ))

    fig.add_trace(go.Scatter(
        x=ML_cumulative_returns.index,
        y=portfolio_normalized,
        mode='lines',
        name='Base Portfolio'
    ))

    fig.update_layout(
        title=f'Comparison of Portfolio vs. Nasdaq Composite Growth (Run {idx})',
        xaxis_title='Date',
        yaxis_title='Normalized Returns (Base 100%)',
        xaxis=dict(type='date', tickformat='%b %Y', tickmode='auto'),
        font=dict(family="Cambria", size=18)
    )

    fig.write_html(f"MLRBAvsRBAvsNasdaq_run{idx}.html")
    # fig.write_image(f"MLRBAvsRBAvsNasdaq_run{idx}.png", format='png', width=1920, height=1080)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Omitted assets: ['BF.B', 'BRK.B']
Time to fetch data: 72.36 seconds
Max combination of assets with complete data: 493
Predicting Week 0 in 100 Total Weeks


Portfolios Tested:   3%|▎         | 1402/50000 [00:02<01:41, 477.57it/s]

All assets have been tested or no improvement possible.
Length of close to best is: 9, Sharpe ratios:[np.float64(2.7966077699764647), np.float64(2.796607769983865), np.float64(2.7970890690141927), np.float64(2.7981566893035827), np.float64(2.872541435004048), np.float64(2.8737353941092105), np.float64(2.877635725694774), np.float64(2.8869932699047927), np.float64(2.8788327400026215)]
RAM Usage: 166.39 MB


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
  2023-01-03 00:00:00: [0.44146922]
  2023-01-04 00:00:00: [0.44101092]
  2023-01-05 00:00:00: [0.4381465]
  2023-01-06 00:00:00: [0.43648896]
  2023-01-09 00:00:00: [0.43548474]
First prediction: [0.44146922] Percentage change: [-1.3555838]
RAM Usage: 121.23 MB
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
  2023-01-03 00:00:00: [0.4925154]
  2023-01-04 00:00:00: [0.49228176]
  2023-01-05 00:00:00: [0.4893091]
  2023-01-06 00:00:00: [0.4881376]
  2023-01-09 00:00:00: [0.48640743]
First prediction: [0.4925154] Percentage change: [-1.2401553]
RAM Usage: 96.09 MB
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
  2023-01-03 00:00:00: [0.47364908]
  2023-01-04 00:00:00: [0.4731124]
  2023-01-05 00:00:00: [0.4698255]
  2023-01-06 00:00:00: [0.46813828]
  2023-01-09 00:00:00: [0.4668308]
First prediction: [0.47364908] Percentage change: [-1.4395244]
RAM Usage: 93.19 MB
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
  2023-01-03 00:00:00: [0.51260066]
  2023-01-04 00:00:00: [0.

...

Portfolios Tested:   3%|▎         | 1380/50000 [00:03<02:01, 401.18it/s]

All assets have been tested or no improvement possible.
Length of close to best is: 4, Sharpe ratios:[np.float64(4.154899627982618), np.float64(4.293611031138071), np.float64(4.312091278724335), np.float64(4.357763773512209)]
RAM Usage: 128.39 MB


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
  2024-12-20 00:00:00: [0.8121387]
  2024-12-23 00:00:00: [0.8049831]
  2024-12-24 00:00:00: [0.79721296]
  2024-12-26 00:00:00: [0.79182625]
  2024-12-27 00:00:00: [0.7969609]
First prediction: [0.8121387] Percentage change: [-1.8688664]
RAM Usage: 128.14 MB
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
  2024-12-20 00:00:00: [0.8185687]
  2024-12-23 00:00:00: [0.8091618]
  2024-12-24 00:00:00: [0.8016014]
  2024-12-26 00:00:00: [0.79418075]
  2024-12-27 00:00:00: [0.7958363]
First prediction: [0.8185687] Percentage change: [-2.7770884]
RAM Usage: 128.16 MB
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
  2024-12-20 00:00:00: [0.83174115]
  2024-12-23 00:00:00: [0.82275677]
  2024-12-24 00:00:00: [0.8161702]
  2024-12-26 00:00:00: [0.80994034]
  2024-12-27 00:00:00: [0.8121761]
First prediction: [0.83174115] Percentage change: [-2.3523]
RAM Usage: 128.22 MB
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
  2024-12-20 00:00:00: [0.83409023]
  2024-12-23 00:00:00: [0.8244399]

Portfolios Tested:   2%|▏         | 1239/50000 [00:02<01:47, 451.98it/s]
/Users/tongchun03/CodeProjects/School/Robo-Advisor/PortfolioPredictor.py:318: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

[*********************100%***********************]  1 of 1 completed

All assets have been tested or no improvement possible.
Held from 2023-01-03 00:00:00 to 2023-01-09 00:00:00: []
Held from 2023-01-10 00:00:00 to 2023-01-17 00:00:00: ['MPC', 'LLY', 'IRM', 'NOC', 'MCK', 'TKO', 'GIS', 'OXY']
Held from 2023-01-18 00:00:00 to 2023-01-24 00:00:00: ['SMCI', 'IRM', 'IT', 'TPL', 'MCK', 'LLY', 'MPC', 'MRNA']
Held from 2023-01-25 00:00:00 to 2023-01-31 00:00:00: ['SMCI', 'IRM', 'IT', 'TPL', 'MCK', 'LLY', 'MPC', 'MRNA']
Held from 2023-02-01 00:00:00 to 2023-02-07 00:00:00: []
Held from 2023-02-08 00:00:00 to 2023-02-14 00:00:00: ['REGN', 'CME', 'DVN', 'MPC', 'MCK', 'AZO', 'LLY', 'HSY']
Held from 2023-02-15 00:00:00 to 2023-02-22 00:00:00: ['NOC', 'NUE', 'SMCI', 'XOM', 'IT', 'AZO', 'MCK', 'GPC']
Held from 2023-02-23 00:00:00 to 2023-03-01 00:00:00: []
Held from 2023-03-02 00:00:00 to 2023-03-08 00:00:00: []
Held from 2023-03-09 00:00:00 to 2023-03-15 00:00:00: []
Held from 2023-03-16 00:00:00 to 2023-03-22 00:00:00: []
Held from 2023-03-23 00:00:00 to 2023-03-29 